In [2]:
import re
import numpy as np
import httpx
import pandas as pd
import json
import os
from datetime import datetime
import ast

In [3]:
def replace_codes_with_chinese(codes, code_dict):
    return [code_dict.get(code, code) for code in codes]


# remove the substring within the last parentheses
def remove_last_parenthesis(s):
    return re.sub(r"\([^)]*\)$", "", s)


def filter_values(series, exclude_values):
    return [x for x in series if pd.notna(x) and x not in exclude_values]


def transformed_to_list(row):
    try:
        return ast.literal_eval(row)
    except:
        return row


def to_list(value):
    if isinstance(value, list):
        return value
    else:
        return [value]

In [4]:
def ingest_to_qdrant(payload: dict):
    url = "http://192.168.1.76:4321/ingest"  # Replace with the appropriate IP and port
    headers = {"Content-Type": "application/json"}
    response = httpx.post(url, json=payload, headers=headers, timeout=600)
    return response

In [7]:
file_columns_df = pd.read_csv("../raw_data/MASS_RUNSQL_20230314_各檔案欄位清單.csv")
file_list_df = pd.read_csv("../raw_data/MASS_RUNSQL_20230314_File_檔案清單.csv")

# we prioritize file with zh name, and column
merged_file_df = pd.merge(file_columns_df, file_list_df, left_on="OBJ_CODE", right_on="FILE_NAME", how="left")
merged_file_df_w_zh_name = merged_file_df.dropna(subset="SP_CNAME").reset_index(drop=True)

# map var_name back to their chinese name
column_list_df = pd.read_csv("../raw_data/MASS_RUNSQL_20230314_Column_欄位清單.csv")

In [8]:
merged_file_df_w_zh_name["SPLIT_VAR_NAME"] = merged_file_df_w_zh_name["VAR_NAME"].str.split(",")
col_name_to_chinese = dict(zip(column_list_df["VAR_NAME"], column_list_df["VAR_CNAME"]))
merged_file_df_w_zh_name["VAR_ZH_NAME"] = merged_file_df_w_zh_name["SPLIT_VAR_NAME"
].apply(lambda x: replace_codes_with_chinese(x, col_name_to_chinese))

# check which value is valid
# via checking if its value in ["是", "否", "不詳", "無", "不明", "其他"]
# or column name does not exist in real column
val_df = pd.read_csv("../raw_data/MASS_20211109_欄位_值對應的中文內容.csv")
val_temp_list = val_df.columns.to_list()
cleaned_val_cols = [remove_last_parenthesis(s) for s in val_temp_list]

/var/folders/8k/5xqs2yln6pbb7bc0prbn30700000gn/T/ipykernel_53160/2817749692.py:9: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,25